**This notebook has been adapted from 'ajc150715_VeecoAFM_CatalystAnalysis.ipynb'

In [1]:
#Loads libs
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pylab as pl
from matplotlib import colors, ticker, cm
from scipy import ndimage
from scipy import optimize 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from math import pi, e
# %matplotlib

Using matplotlib backend: MacOSX


In [2]:
#To configure plot labels
def configPlot(xAxis='',yAxis='',fileTitle=''):
    plt.xlabel(xAxis,fontsize=20)
    plt.ylabel(yAxis,fontsize=20)
    plt.title(fileTitle)

In [ ]:
#VARIABLES FOR DETERMINE CENTERS OF CATALYSTS
smooth = 5 #Lower smoothness for less smoothing and increase for more
colorRes = 100 #for plot
plotHighT = 10 #for plot


#
# --------------------//-----------------------------
#

#VARIABLES FOR GET CATALYST HEIGHT DISTRIBUTION
maxNB = 5 #neighborhood to search for max
minNB = 50 #neighborhood to search for mins
lowThreshold = 0.
histBins = 100
histMin = 0
histMax = 10

#
# --------------------//-----------------------------
#

#VARIABLES FOR PLOT EXAMPLES OF CATALYST HEIGHT EXTRACTION
nbPlot = 100 #pixel neighborhood to plot
numX = 5
numY = 5

# Catalyst Size Analysis

## Local Maximum Finder

In [3]:
# Source: http://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array/3689710#3689710
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion

def detect_peaks(image):
    """
    Takes an image and detect the peaks using the local maximum filter.
    Returns a boolean mask of the peaks (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """

    # define an 8-connected neighborhood
    neighborhood = generate_binary_structure(2,2)

    #apply the local maximum filter; all pixel of maximal value 
    #in their neighborhood are set to 1
    local_max = maximum_filter(image, footprint=neighborhood)==image
    #local_max is a mask that contains the peaks we are 
    #looking for, but also the background.
    #In order to isolate the peaks we must remove the background from the mask.

    #we create the mask of the background
    background = (image==0)

    #a little technicality: we must erode the background in order to 
    #successfully subtract it form local_max, otherwise a line will 
    #appear along the background border (artifact of the local maximum filter)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)

    #we obtain the final mask, containing only peaks, 
    #by removing the background from the local_max mask
    detected_peaks = local_max - eroded_background

    return detected_peaks

## Determine Centers of Catalysts

In [4]:
def plotCatMap(fn,colorResolution=200,plotHighThreshold=10,smoothness=2,filetype='png'):
    plt.rcParams.update({'font.size': 15})
    
    #Loads data
    data = np.loadtxt(fn,skiprows=1)

    #Load in catalyst heights then reshape into a matrix
    CHeights = np.copy(data[:,0])
    CHeights = CHeights.reshape(32,16384)
    
#     #Zoom into region
#     CHeights = CHeights[0:17,400:2700]

    #Get dimensions of x and y 
    nx = np.shape(CHeights)[0]
    ny = np.shape(CHeights)[1]

    CHeightsAdj = np.copy(CHeights)

    #Plot adjusted heights with a high threshold 
    fig = plt.figure(1)
    ax = plt.subplot(111)
    CHeightsAdjHighThres = np.copy(CHeightsAdj)
    CHeightsAdjHighThres[CHeightsAdj>plotHighThreshold] = plotHighThreshold
    PC = plt.contourf(CHeightsAdjHighThres,colorResolution)
    ax.set_aspect(128)
    
    # Adding the colorbar
    cb = plt.colorbar(fraction=0.03,aspect=10)  
    cb.set_label('  '+ ' nm',rotation=0)

    #Smooths out image for peak finding algorithm only in direction
    #Lower sigma for less smoothing and increase for more averaging
    for i in range(nx):
        CHeightsAdj[i,:] = ndimage.gaussian_filter(CHeightsAdj[i,:], sigma=smoothness)

    #Uses peak detection algorithm
    result = detect_peaks(CHeightsAdj)*1.0
    peaks = np.where(result==1)
    peaksX = peaks[1]
    peaksY = peaks[0]

    pl.plot(peaksX,peaksY,marker='o',markersize=1.,linestyle='',color='r')
    
    #Saving file as vector PDF or highres PNG
    fnTitle = fn.replace('.txt','')
    if filetype=='pdf':
        plt.savefig(fnTitle + '_Heights.pdf', format='PDF')
    elif filetype=='png':
        plt.savefig(fnTitle + '_Heights.png',dpi=500)
        
    return CHeights,CHeightsAdj,peaksX,peaksY

## Get Catalyst Height Distribution

In [5]:
def plotCatDist(CHeights,CHeightsAdj,peaksX,peaksY,maxNB=10,minNB=50,lowThreshold=0.5,histBins=50,histMin=1,histMax=30,filetype='png'):
    plt.rcParams.update({'font.size': 20})
    
    peakHeights = np.array([])
    ny = np.shape(CHeights)[0]
    nx = np.shape(CHeights)[1]
    
    #included peaks dictionary
    includedPeaks = []
    
    for i in range(len(peaksX)):
        locX = peaksX[i]
        locY = peaksY[i]
        maxRange = CHeights[locY,locX - maxNB:locX + maxNB]
        minRangeL = CHeights[locY,locX - minNB:locX]
        minRangeR = CHeights[locY,locX:locX + minNB]
        if len(maxRange) > 0 and len(minRangeL) > 0 and len(minRangeR) > 0:
            maxH = max(maxRange)
            minHL = min(minRangeL)
            minHR = min(minRangeR)
            h = maxH - (minHL + minHR)/2.
            if h > lowThreshold:
                peakHeights = np.append(peakHeights,h)
                
                includedPeak = ({'locX':locX,
                                      'locY':locY,
                                      'maxH':maxH,
                                      'minHL':minHL,
                                      'minHR':minHR,
                                      'h':h
                                      })
                includedPeaks.append(includedPeak)
                
    fig = plt.figure(2)   
    plt.hist(peakHeights,bins=histBins,range=(histMin,histMax));
    configPlot('Height (nm)','Counts')
    
    #Saving file as vector PDF or highres PNG
    fnTitle = fn.replace('.txt','')
    if filetype=='pdf':
        plt.savefig(fnTitle + '_Dist.pdf', format='PDF')
    elif filetype=='png':
        plt.savefig(fnTitle + '_Dist.png',dpi=500)
        
    return includedPeaks

## Plot Examples of Catalyst Height Extraction

In [6]:
def plotCatExamples(CHeights,CHeightsAdj,includedPeaks,numX=2,numY=5,nbPlot=200,filetype='png'):

    plt.rcParams.update({'font.size': 9})
    
    f, axarr = plt.subplots(numX, numY)
    randomPeaks = np.random.choice(range(len(includedPeaks)),size=numX*numY,replace=False)

    count = 0
    for i in range(numY):
        for j in range(numX):
            peak = includedPeaks[randomPeaks[count]]
            locX = peak.get('locX')
            locY = peak.get('locY')
            y = CHeights[locY,locX - nbPlot/2:locX + nbPlot/2]
            y2 = CHeightsAdj[locY,locX - nbPlot/2:locX+nbPlot/2]
            x = range(len(y))
            center = x[len(y)//2]
            maxH = y[center]
            axarr[j,i].plot(x,y,x,y2)
            h = peak.get('h')
            axarr[j,i].plot([center,center], [maxH, maxH - h], color='red', linestyle='-', linewidth=1)
            count += 1
            
    f.subplots_adjust(hspace=0.5,wspace=0.5)
    
    #Saving file as vector PDF or highres PNG
    fnTitle = fn.replace('.txt','')
    if filetype=='pdf':
        plt.savefig(fnTitle + '_Examples.pdf', format='PDF')
    elif filetype=='png':
        plt.savefig(fnTitle + '_Examples.png',dpi=500)

## Run Catalyst Analysis

In [8]:
# #VARIABLES FOR DETERMINE CENTERS OF CATALYSTS
# smooth = 5 #Lower smoothness for less smoothing and increase for more
# colorRes = 100 #for plot
# plotHighT = 10 #for plot


# #
# # --------------------//-----------------------------
# #

# #VARIABLES FOR GET CATALYST HEIGHT DISTRIBUTION
# maxNB = 5 #neighborhood to search for max
# minNB = 50 #neighborhood to search for mins
# lowThreshold = 0.
# histBins = 100
# histMin = 0
# histMax = 10

# #
# # --------------------//-----------------------------
# #

# #VARIABLES FOR PLOT EXAMPLES OF CATALYST HEIGHT EXTRACTION
# nbPlot = 100 #pixel neighborhood to plot
# numX = 5
# numY = 5

# #
# # --------------------//-----------------------------
# #

# # Load in AFM text file
# # os.chdir("C:/Users/Joshua/Documents/Nanotubes/AFMimages/150716/Ascii")
# fn = '../data_vault/AFM/150716_W10_Catalystlines-000.txt'

# #Run Catalyst Peak Finder
# CHeights,CHeightsAdj,peaksX,peaksY= plotCatMap(fn,
#                                                 colorResolution=colorRes,
#                                                 plotHighThreshold=plotHighT,
#                                                 smoothness=smooth,
#                                                 filetype='png')

# #Plot the catalyst size distribution
# includedPeaks = plotCatDist(CHeights,CHeightsAdj,peaksX,peaksY,
#             maxNB=maxNB,
#             minNB=minNB,
#             lowThreshold=lowThreshold,
#             histBins=histBins,
#             histMin=histMin,
#             histMax=histMax,
#             filetype='png')

# #Plot a few examples of catalyst analysis
# plotCatExamples(CHeights,CHeightsAdj,
#                 includedPeaks,
#                 nbPlot=nbPlot,
#                 numX=numX,
#                 numY=numY,
#                 filetype='png')

In [45]:
# # defining fitting function
# def gau_fit(histrange, heights):
#     return 1/(2 * pi * pstdev(heights)) * (e*np.exp(-((histBins-mean(heights))**2) / (2*pstdev(heights))**2))

In [47]:
# # Curve fit
# par, covar = opt.curve_fit(gau_fit, histBins, includedPeaks['h'], [1,6])

# #plotting results
# fig = plt.figure(4)
# pl.plot(gau_fit(880, par))

TypeError: list indices must be integers, not str